# Filtering galaxies

### We exclude galaxies which have already been observed in former missions. For future use of this code, it needs to be updated with new sources that were observed.

In [1]:
import astropy.units as u
import pandas as pd
import numpy as np
from astropy.coordinates import Angle

In [27]:
# You need to give a list of astronomical objects with spherical coordinates RA & DEC
df = pd.read_csv('/home/pampolha/Observações/S4G_1.csv')
mask_ra = (df['ra'] > 100) & (df['ra'] < 330)
mask_dec = (df['dec'] > (-60)) & (df['dec'] < 22)
mask_mstar = (df['mstar'] > 10) #MW analogs criteria
mask_vmaxg = (df['vmaxg'] > 100) #MW analogs criteria
mask_t = (df['t'] > 1.5) & (df['t'] < 4.5) #MW analogs criteria
mask_incl = (df['incl'] > 40) & (df['incl'] < 70) #MW analogs criteria

# Combine masks using the logic operator "&" (AND)
mask_combined = mask_ra & mask_dec & mask_mstar & mask_vmaxg & mask_t & mask_incl

# Remove lines that doesn't satisfy all criteria at the same time
df = df[mask_combined]

# List of galaxies to be removed (previous missions objects + actual)

# Loads the main table and add new columns
df['equinox']='J2000'

# To upload the coordinates to Staralt, RA must be at degree units 
df['ra_hours'] = df['ra'].apply(lambda x: (x * u.deg).to(u.hourangle).value)

# Lists the galaxies to be removed (past missions + current)
galaxias_remover = ['ESO404-012','NGC5750','NGC5757','NGC4045','IC4237', 'NGC5957','NGC5962','NGC3321', 
                    'IC1055','IC1067','IC1158','IC1438','NGC4897','NGC4902','NGC5913','NGC7219','NGC7690',
                    'NGC4480','NGC4378','NGC4409','NGC4580','PGC036217','NGC7421','NGC6870','NGC6902',
                    'NGC6923','NGC4941','NGC4050','NGC7418','NGC6118','IC4901','NGC6887','NGC6063',
                    'NGC6925','NGC6070','NGC2731','NGC3507','NGC7205','NGC4158','NGC2966','NGC4527','NGC4351',
                    'NGC5247','NGC5756','NGC7552','NGC7179','NGC3976','NGC4628','NGC4067','NGC5364','NGC4313',
                    'UGC08067','NGC3715','NGC3177','ESO443-042','NGC7368','NGC4639','IC5334','NGC4343','PGC047721',
                    'NGC4394','NGC3681','NGC4462','NGC4746','NGC7171','NGC5534','NGC3705','NGC3684', 'NGC5806',
                    'NGC5850', 'NGC5878','NGC5915','NGC5921','NGC5937','NGC7163','NGC6012','NGC7140','ESO342-013']

# Filters DataFrame, keeping only galaxies whiche are NOT on remotion list
df_filtered = df[~df['object'].isin(galaxias_remover)]

# Shows how many galaxies were removed
print(f"Original: {len(df)}")
print(f"Total after filter: {len(df_filtered)}")
print(f"# of removed galaxies: {len(df) - len(df_filtered)}")

# Saves the result into a .csv file
df_filtered.to_csv('tabela_filtrada.csv', index=False)

Original 22
Total after filter: 14
# of removed galaxies: 8


Selecting only the galaxy position columns

In [3]:
df_positions = df_filtered[['object', 'ra','dec']]
df_positions

,object,ra,dec
1,IC5039,310.809676,-29.853357
2,IC5105B,321.500887,-40.834792
3,ESO234-049,308.825250,-49.865630
4,ESO285-048,311.167157,-45.978448
7,NGC6889,304.721395,-53.956999
10,NGC7059,321.839586,-60.014545
12,NGC7151,328.766150,-50.657610
13,NGC7162,329.912629,-43.306085
15,PGC066242,317.521256,-3.701667
16,ESO402-025,320.566343,-36.627557


In [29]:
# Display the first three columns of the filtered DataFrame (to use on Finders.ipynb notebook)
print(df_filtrado.iloc[:, :3].to_string(index=False)).drop

    object         ra        dec
    IC5039 310.809676 -29.853357
   IC5105B 321.500887 -40.834792
ESO234-049 308.825250 -49.865630
ESO285-048 311.167157 -45.978448
   NGC6889 304.721395 -53.956999
   NGC7059 321.839586 -60.014545
   NGC7151 328.766150 -50.657610
   NGC7162 329.912629 -43.306085
 PGC066242 317.521256  -3.701667
ESO402-025 320.566343 -36.627557
ESO403-024 326.665209 -36.205534
ESO403-031 327.016853 -36.772655
ESO404-003 328.165578 -36.502734
ESO342-050 322.062601 -37.861329


AttributeError: 'NoneType' object has no attribute 'drop'

### Creates a table with these positions

In [5]:
df_final.to_csv('staralt.csv')

# Converting coordinates from deg deg to h m s and deg m s

### To set coordinates of targets at the Pointing, in TCSPD monitor, those should be on the units
##### ra: hour minutes seconds
##### dec: degree minutes seconds
### Let´s make the units conversion to all of them

In [25]:
df_final['ra_hms'] = df_final['ra'].apply(
    lambda x: Angle(x, unit='deg').to_string(unit='hour', sep=':', pad=True))

# Converting only decimals to DMS
def decimal_to_dms(decimal_deg):
    degrees = int(decimal_deg)  # Full part of degrees
    decimal_part = abs(decimal_deg - degrees)  # Decimal part (0.xxx)
    
    # Converts decimal part to minutes and seconds
    total_seconds = decimal_part * 3600  # 1° = 3600 seconds
    minutes = int(total_seconds // 60)
    seconds = total_seconds % 60
    
    return f"{degrees}° {minutes}' {seconds:.2f}\""

# Aplica a função à coluna
df_final['dec_dms'] = df_final['dec'].apply(decimal_to_dms)

df_f = df_final[['object', 'ra_hms', 'dec_dms']]

/tmp/ipykernel_310/2140192615.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['ra_hms'] = df_final['ra'].apply(
/tmp/ipykernel_310/2140192615.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['dec_dms'] = df_final['dec'].apply(decimal_to_dms)


In [26]:
df_f

,object,ra_hms,dec_dms
1,IC5039,20:43:14.32224,"-29° 51' 12.09"""
2,IC5105B,21:26:00.21288,"-40° 50' 5.25"""
3,ESO234-049,20:35:18.06,"-49° 51' 56.27"""
4,ESO285-048,20:44:40.11768,"-45° 58' 42.41"""
7,NGC6889,20:18:53.1348,"-53° 57' 25.20"""
10,NGC7059,21:27:21.50064,"-60° 0' 52.36"""
12,NGC7151,21:55:03.876,"-50° 39' 27.40"""
13,NGC7162,21:59:39.03096,"-43° 18' 21.91"""
15,PGC066242,21:10:05.10144,"-3° 42' 6.00"""
16,ESO402-025,21:22:15.92232,"-36° 37' 39.21"""


In [24]:
#Saves the updated coordinates into a csv file
df_final.to_csv('staralt_hours.csv')